<a href="https://colab.research.google.com/github/LorenaVilaca/CharMix/blob/main/notebooks/CharMix_Requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CharMaker

#### **Flow**
1. Imports
2. Preparo das imagens
- [X] ~~Testar carregar as imagens base64~~
- [X] ~~Testar carregar com URL~~
3. Envio de primeiro prompt

```
You are a description assistant, skilled in describing an image to others who can't see it.
You must structure your description in six main categories:
Appearance, Facial expression, Body pose, Clothes, The feeling the image gives and the Artstyle (including if it is 2D or 3D).

```
4. Envio das imagens -> Primeiro descreva uma e depois a outra
- [X] ~~Criar variável pro prompt~~
- [X] ~~Checar se o envio ta funcionando~~
- [X] ~~Extrair e salvar descrição da response~~ (talvez em algum arquivo) -> Solução: variável


5. Mistura das descrições

6. Envio da nova descrição pra geração de imagem
- [X] ~~Gerar URL~~
- [X] ~~Testar fazer o donwload do arquivo~~

```
You are a description assistant, skilled in combining the descriptions of two different images in one new character with mixed characteristics.
You must structure your description in six main categories:
APPEARENCE, FACIAL EXPRESSION, BODY POSE, CLOTHES, THE FELLING THE IMAGE GIVES and ARTSTYLE.
Combine these two descriptions, and the new Artstyle should be the same as the first description's Artstyle.
```

#### **Dúvidas**

* O chatgpt retém informações tbm pela api? No sentido de dar pra continuar uma conversa -> Aparentemente [não](https://platform.openai.com/docs/guides/vision#:~:text=The%20Chat%20Completions%20API%2C%20unlike%20the%20Assistants%20API%2C%20is%20not%20stateful)
* Depois tem que fazer a parte de tratar as descrições recebidas pra filtrar etc


#### **Ferramentas**

* Dalle
* ChatGPT

#### **REF**
* [Vision API Doc](https://platform.openai.com/docs/guides/vision)
---


## Setup Inicial

In [4]:
!pip install openai

In [5]:
import os
from google.colab import userdata
from openai import OpenAI

# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api')

client = OpenAI(api_key = userdata.get('openai_api'))

modelo_prompt_descricao = """
You are a description assistant, skilled in describing an image to others who can't see it.
You must structure your description in six main categories:
Appearance, Facial expression, Body pose, Clothes, The feeling the image gives and the Artstyle (including if it is 2D or 3D).
"""

modelo_prompt_geracao = """
You are a description assistant, skilled in combining the descriptions of two different images in one new character with mixed characteristics.
You must structure your description in six main categories:
APPEARENCE, FACIAL EXPRESSION, BODY POSE, CLOTHES, THE FELLING THE IMAGE GIVES and ARTSTYLE.
Combine these two descriptions, and the new Artstyle should be the same as the first description's Artstyle.
"""

result_path = "/content/THIS_IS_THE_RESULT.jpg"

## Preparo das imagens

**Rodar apenas 1, dependendo da opção.**

#### Com imagem local

Colocar as imagens na pasta *input_images* e trocar as variáveis "*image_path*" pra elas

In [7]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to images
image1_path = "/content/input_images/coraline.png"
image2_path = "/content/input_images/olivia_palito.jpg"

# Getting the base64 string
base64_image1 = encode_image(image1_path)
base64_image2 = encode_image(image2_path)

# Formatting
image1 = f"data:image/jpeg;base64,{base64_image1}"
image2 = f"data:image/jpeg;base64,{base64_image2}"

#### Com URL
Neste caso, só mudar as urls aqui embaixo

In [ ]:
image1 = "https://64.media.tumblr.com/75d11ac15910572e6746076a5f85ee79/tumblr_inline_p7hjyx0kY11spo3o7_500.jpg" # Wirt
image2 = "https://i0.wp.com/otakusbrasil.com/wp-content/uploads/2023/10/Sailor_Moon_TV_Series-848012693-large.jpg" # Sailor Moon

## Envio das imagens

In [8]:
def get_description(image_ref):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": modelo_prompt_descricao},
          {
            "type": "image_url",
            "image_url": {
              "url": image_ref
  ,
            },
          },
        ],
      }
    ],
    max_tokens=500,
  )
  return response.choices[0].message.content

In [10]:
# ta feio mas tbh to fazendo de qualquer jeito pra funcionar primeiro

description_1= get_description(image1)
description_2= get_description(image2)

descriptions = f"First character:\n{description_1}\n\nSecond character:\n{description_2}"

# print (descriptions)


## Geração da nova imagem

In [11]:
response = client.images.generate(
  model="dall-e-3",
  prompt=f"{modelo_prompt_geracao} \n {descriptions}",
  n=1,
  response_format = "b64_json",
  quality="standard",
  size="1024x1024"
)

image_data= response.data[0]

'''
---> Pra pegar a URL ao invés de pegar o arquivo
* remover linha "response_format = b64_json" do request
* descomentar linhas abaixo

Lembrando que URLs só são válidas por 60 minutos pós-criação
'''
#image_url = image_data.url
#print(image_url)


'\n---> Pra pegar a URL ao invés de pegar o arquivo\n* remover linha "response_format = b64_json" do request\n* descomentar linhas abaixo\n\nLembrando que URLs só são válidas por 60 minutos pós-criação\n'

In [12]:
from PIL import Image
from io import BytesIO

image_obj = image_data.model_dump()["b64_json"]
image_obj = Image.open(BytesIO(base64.b64decode(image_obj)))
image_obj.save("/content/THIS_IS_THE_RESULT.jpg")